# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from html.parser import HTMLParser

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# clean up cities remove []
cities['NHL'] = cities['NHL'].str.replace(r"\s*\[.*?\]\s*", '').str.strip()

#clean up cities remove no team or '-'
cities = cities[cities.NHL != '—']

#clean up Red Wings
cities = cities.replace({"Red Wings": "Redwings", "Maple Leafs": "Mapleleafs", "Golden Knights": "Goldenknights", "Blue Jackets": "Bluejackets" }, regex=True)

#clean up cities - separate by upper case letters 
cities['NHL'] = cities['NHL'].str.replace(r'(?<!^)([A-Z])', r' \1')

# cities new rows for each team ()
cities = cities.assign(NHL=cities['NHL'].str.split(' ')).explode('NHL')

#remove rows with no NHL values
cities = cities[cities.NHL != '']


#clean-up nhl team names - grab last word

#nhl_df - filter for 2018 data
nhl_df = nhl_df[nhl_df.year == 2018]
nhl_df['team'] = nhl_df['team'].str.replace('*', '')
nhl_df = nhl_df.replace({"Red Wings": "Redwings", "Maple Leafs": "Mapleleafs", "Golden Knights": "Goldenknights", "Blue Jackets": "Bluejackets" }, regex=True)
nhl_df['team'] = nhl_df['team'].str.split().str[-1]
nhl_df = nhl_df[nhl_df.team != 'Division']
#nhl_df - new column - win/loss ratio

nhl_df['Ratio'] = pd.to_numeric(nhl_df['W'])/ (pd.to_numeric(nhl_df['W']) + pd.to_numeric(nhl_df['L']))
#merge dataframe by team

#df2_tidy = df2_melted.rename(columns = {'variable': 'Year', 'value': 'Income'}, inplace = False)

cities = cities.rename(columns = {'NHL': 'team'})

df = pd.merge(cities, nhl_df, how='inner', on='team')
df = df.rename(columns = {'Population (2016 est.)[8]': 'Population'})
df['Population'] = pd.to_numeric(df['Population'])
#pivot table index = metro, mean of win/loss ratio

pop_table = pd.pivot_table(df, values='Population', index='Metropolitan area', aggfunc='mean')
ratio_table = pd.pivot_table(df, values='Ratio', index='Metropolitan area', aggfunc='mean')
 
def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pop_table['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = ratio_table['Ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    

    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return(corr) 

nhl_correlation()


0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# clean up cities remove []
cities['NBA'] = cities['NBA'].str.replace(r"\s*\[.*?\]\s*", '').str.strip()

# #clean up cities remove no team or '-'
cities = cities[cities.NBA != '—']

# #clean up Red Wings
cities = cities.replace({"Trail Blazers": "Trailblazer"}, regex=True)

# #clean up cities - separate by upper case letters 
cities['NBA'] = cities['NBA'].str.replace(r'(?<!^)([A-Z])', r' \1')

# # cities new rows for each team ()
cities = cities.assign(NBA=cities['NBA'].str.split(' ')).explode('NBA')

# #remove rows with no NBA values
cities = cities[cities.NBA != '']
cities = cities.rename(columns = {'NBA': 'team'})


# clean-up nhl team names - grab last word

#nba_df - filter for 2018 data
nba_df = nba_df[nba_df.year == 2018]
nba_df = nba_df.replace({"Trail Blazers": "Trailblazer"}, regex=True)

nba_df['team'] = nba_df['team'].str.replace('*', '')
nba_df['team'] = nba_df['team'].str.replace(r"\(.*\)", '').str.strip()

nba_df['team'] = nba_df['team'].str.split().str[-1]

nba_df['Ratio'] = pd.to_numeric(nba_df['W'])/ (pd.to_numeric(nba_df['W']) + pd.to_numeric(nba_df['L']))


df = pd.merge(cities, nba_df, how='inner', on='team')
df = df.rename(columns = {'Population (2016 est.)[8]': 'Population'})
df['Population'] = pd.to_numeric(df['Population'])

#pivot table index = metro, mean of win/loss ratio

pop_table = pd.pivot_table(df, values='Population', index='Metropolitan area', aggfunc='mean')
ratio_table = pd.pivot_table(df, values='Ratio', index='Metropolitan area', aggfunc='mean')
 

def nba_correlation():
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pop_table['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = ratio_table['Ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return(corr)  

nba_correlation()

-0.17657160252844617

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [8]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# clean up cities remove []
cities['MLB'] = cities['MLB'].str.replace(r"\s*\[.*?\]\s*", '').str.strip()

# #clean up cities remove no team or '-'
cities = cities[cities.MLB != '—']

# # #clean up Red Wings
cities = cities.replace({"Blue Jays": "Bluejays", "White Sox": "Whitesox", "Red Sox": "Redsox"}, regex=True)

# # #clean up cities - separate by upper case letters 
cities['MLB'] = cities['MLB'].str.replace(r'(?<!^)([A-Z])', r' \1')

# # # cities new rows for each team ()
cities = cities.assign(MLB=cities['MLB'].str.split(' ')).explode('MLB')

# # #remove rows with no NBA values
cities = cities[cities.MLB != '']
cities = cities.rename(columns = {'MLB': 'team'})


# # clean-up nhl team names - grab last word

# #nba_df - filter for 2018 data
mlb_df = mlb_df[mlb_df.year == 2018]
mlb_df = mlb_df.replace({"Blue Jays": "Bluejays", "White Sox": "Whitesox", "Red Sox": "Redsox"}, regex=True)

mlb_df['team'] = mlb_df['team'].str.replace('*', '')
mlb_df['team'] = mlb_df['team'].str.replace(r"\(.*\)", '').str.strip()

mlb_df['team'] = mlb_df['team'].str.split().str[-1]

mlb_df['Ratio'] = pd.to_numeric(mlb_df['W'])/ (pd.to_numeric(mlb_df['W']) + pd.to_numeric(mlb_df['L']))


df = pd.merge(cities, mlb_df, how='inner', on='team')
df = df.rename(columns = {'Population (2016 est.)[8]': 'Population'})
df['Population'] = pd.to_numeric(df['Population'])

# #pivot table index = metro, mean of win/loss ratio

pop_table = pd.pivot_table(df, values='Population', index='Metropolitan area', aggfunc='mean')
ratio_table = pd.pivot_table(df, values='Ratio', index='Metropolitan area', aggfunc='mean')
 
def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pop_table['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = ratio_table['Ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return(corr) 
mlb_correlation()


0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# clean up cities remove []
cities['NFL'] = cities['NFL'].str.replace(r"\s*\[.*?\]\s*", '').str.strip()

# # #clean up cities remove no team or '-'
cities = cities[cities.NFL != '—']

# # # #clean up Red Wings

# # # #clean up cities - separate by upper case letters 
cities['NFL'] = cities['NFL'].str.replace(r'(?<!^)([A-Z])', r' \1')

# # # # cities new rows for each team ()
cities = cities.assign(NFL=cities['NFL'].str.split(' ')).explode('NFL')

# # # #remove rows with no NBA values
cities = cities[cities.NFL != '']
cities = cities.rename(columns = {'NFL': 'team'})


# # # clean-up nhl team names - grab last word

# # #nba_df - filter for 2018 data
nfl_df = nfl_df[nfl_df.year == 2018]
nfl_df['team'] = nfl_df['team'].str.replace('*', '')
nfl_df['team'] = nfl_df['team'].str.replace(r"\(.*\)", '').str.strip()
nfl_df['team'] = nfl_df['team'].str.split().str[-1]
nfl_df = nfl_df[nfl_df.team != 'North']
nfl_df = nfl_df[nfl_df.team != 'East']
nfl_df = nfl_df[nfl_df.team != 'South']
nfl_df = nfl_df[nfl_df.team != 'West']
nfl_df['Ratio'] = pd.to_numeric(nfl_df['W'])/ (pd.to_numeric(nfl_df['W']) + pd.to_numeric(nfl_df['L']))


df = pd.merge(cities, nfl_df, how='inner', on='team')
df = df.rename(columns = {'Population (2016 est.)[8]': 'Population'})
df['Population'] = pd.to_numeric(df['Population'])

# # #pivot table index = metro, mean of win/loss ratio

pop_table = pd.pivot_table(df, values='Population', index='Metropolitan area', aggfunc='mean')
ratio_table = pd.pivot_table(df, values='Ratio', index='Metropolitan area', aggfunc='mean')
 
def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    
    population_by_region = pop_table['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = ratio_table['Ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    
    corr, pval = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return(corr/3) 
nfl_correlation()

0.012859868107168096

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [12]:
mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")


def nhl_df():
    Big4 = 'NHL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—",np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    _df = pd.read_csv("assets/" + str.lower(Big4) + ".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'\*', "")
    _df = _df[['team', 'W', 'L']]

    dropList = []
    for i in range(_df.shape[0]):
        row = _df.iloc[i]
        if row['team'] == row['W'] and row['L'] == row['W']:
            dropList.append(i)
    _df = _df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ', '')
    _df = _df.astype({'team': str, 'W': int, 'L': int})
    _df['W/L%'] = _df['W'] / (_df['W'] + _df['L'])

    merge = pd.merge(team, _df, 'inner', on='team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]


def nba_df():
    Big4 = 'NBA'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—",np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    _df = pd.read_csv("assets/" + str.lower(Big4) + ".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]', "")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)', "")
    _df['team'] = _df['team'].str.replace(r'[\xa0]', "")
    _df = _df[['team', 'W/L%']]
    _df['team'] = _df['team'].str.replace('[\w.]* ', '')
    _df = _df.astype({'team': str, 'W/L%': float})

    merge = pd.merge(team, _df, 'outer', on='team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]


def mlb_df():
    Big4 = 'MLB'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—",np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('\ Sox', 'Sox')
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    _df = pd.read_csv("assets/" + str.lower(Big4) + ".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]', "")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)', "")
    _df['team'] = _df['team'].str.replace(r'[\xa0]', "")
    _df = _df[['team', 'W-L%']]
    _df.rename(columns={"W-L%": "W/L%"}, inplace=True)
    _df['team'] = _df['team'].str.replace('\ Sox', 'Sox')
    _df['team'] = _df['team'].str.replace('[\w.]* ', '')
    _df = _df.astype({'team': str, 'W/L%': float})

    merge = pd.merge(team, _df, 'outer', on='team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]


def nfl_df():
    Big4 = 'NFL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area'] = cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—",np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    _df = pd.read_csv("assets/" + str.lower(Big4) + ".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]', "")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)', "")
    _df['team'] = _df['team'].str.replace(r'[\xa0]', "")
    _df = _df[['team', 'W-L%']]
    _df.rename(columns={"W-L%": "W/L%"}, inplace=True)
    dropList = []
    for i in range(_df.shape[0]):
        row = _df.iloc[i]
        if row['team'] == row['W/L%']:
            dropList.append(i)
    _df = _df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ', '')
    _df['team'] = _df['team'].str.replace('+', '')
    _df = _df.astype({'team': str, 'W/L%': float})

    merge = pd.merge(team, _df, 'outer', on='team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]


def create_df(sport):
    if sport == 'NFL':
        return nfl_df()
    elif sport == 'NBA':
        return nba_df()
    elif sport == 'NHL':
        return nhl_df()
    elif sport == 'MLB':
        return mlb_df()
    else:
        print("ERROR with intput!")


def sports_team_performance():
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k: np.nan for k in sports}, index=sports)

    for i in sports:
        for j in sports:
            if i != j:
                merge = pd.merge(create_df(i), create_df(j), 'inner', on=['Metropolitan area'])
                p_values.loc[i, j] = stats.ttest_rel(merge['W/L%_x'], merge['W/L%_y'])[1]

    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values



In [13]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


In [18]:
import numpy as np

a = np.arange(8)
b = a[4:6]
b[:] = 40
c = a[4] + a[6]
print(c)

46
